<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/two_by_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Two-by-Two Encoding

This notebook shows that every 1# program may be modified to get a program with the special feature that when it runs, the case statement 1##### is never called on an empty R1.  (This technical feature will be put to use later, when we prove the undecidability of Post's Correspondence Problem).

The work here also is a good illustration of the use of the Sanity too.

In [ ]:
#@title
!python -m pip install -U setuptools
!python -m pip install -U git+https://github.com/lmoss/onesharp/
from onesharp.interpreter.interpreter import *
from onesharp.tools.sanity import *

The two-by-two encoding takes the symbol 1 and replaces it by 11, and the symbol # and replaces it by 1#.  Then a word in a given register is replace by the letter-by-letter encoding, followed by ## at the end of the register.   For example, if R1 has 1#1, then we encode this by 111#11##.  
We want to translate programs into ones which work on the encoded words, and we want to be sure that the translated programs have the feature that we are looking for. The basic idea in the programs below is that we can grab the symbols in R1 in a `two-by-two' fashion, and since we have an `end-of-register' marker ##, we will never do a case statement on an empty register.

In [41]:
two_by_two_addone_idea = [ # this is sanity code for 1#, run on encoded R1
    ['top', 'cases', 1, 'empty', 'found_a_one', 'found_a_sharp'],
    ['empty','1###'],
    ['found_a_one', 'cases', 1, 'empty', 'one_one', 'one_sharp'],
    ['found_a_sharp', 'cases', 1, 'empty', 'empty', 'sharp_sharp'],
    ['one_one', '1#1#'],
    ['goto', 'top'],
    ['one_sharp', '1#1##'],
    ['goto', 'top'],
    ['sharp_sharp', '1#1#1##1##'], ## note that the 1#1# adds 11 before the end
    ['goto', 'end']
]

two_by_two_addone = sanity(two_by_two_addone_idea)

two_by_two_addsharp_idea = [ # this is sanity code for 1##, again run on encoded R1
    ['top', 'cases', 1, 'empty', 'found_a_one', 'found_a_sharp'],
    ['empty','1###'],
    ['found_a_one', 'cases', 1, 'empty', 'one_one', 'one_sharp'],
    ['found_a_sharp', 'cases', 1, 'empty', 'empty', 'sharp_sharp'],
    ['one_one', '1#1#'],
    ['goto', 'top'],
    ['one_sharp', '1#1##'],
    ['goto', 'top'],
    ['sharp_sharp', '1#1##1##1##'], ## note that the 1#1## adds 11 before the end
    ['goto', 'end']
]

two_by_two_addone = sanity(two_by_two_addone_idea)

two_by_two_addsharp = sanity(two_by_two_addsharp_idea)

In [15]:
onesharp(two_by_two_addsharp,['1###'])

'1#1###'

In [94]:
def replace_add_instruction(x):
  if x[1] == '1#':
    return([x[0], two_by_two_addone])
  elif  x[1] == '1##':
    return([x[0], two_by_two_addsharp]) 
  else:
    return(x)


In [43]:
A = parse('11###1#')
B = [[x] for x in A]
C = [replace_add_instruction(c) for c in B]
print(B)
print(C)
print(sanity(C))
onesharp(sanity(C),['11##'])

[['11###'], ['1#']]
[['11###'], ['1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###']]
11###1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###


KeyboardInterrupt: ignored

In [95]:
def numbered(parsed): ## words on a parsed program
  n = len(parsed)
  x = [[str(i),parsed[i]] for i in range(n)]
  return(x)

def resolve_transfer(instruction): # for use on an instruction in a numbered, parsed program  
  m = int(instruction[0])
  if instruction_type(instruction[1]) == 'forward':
     offset = len(instruction[1]) - 3
     return([str(m), 'goto', int(m+offset)])
  if instruction_type(instruction[1]) == 'backward':
     offset = len(instruction[1]) - 4
     return([str(m), 'goto', int(m-offset)])     
  else:
     return(instruction)   

def resolve_all_transfers(parsed): #for use on a parsed, numbered program 
  answer = [resolve_transfer(i) for i in parsed]
  return(answer)

def replace_all_add_instructions(parsed): #for use on a parsed, numbered program 
  answer = [replace_add_instruction(i) for i in parsed]
  return(answer)

In [116]:
pzero = '1##1#'
pone = parse(pzero)
print(pone)
p = numbered(pone)
print(p)
q = resolve_all_transfers(p)
print('q below')
print(q)
r =replace_all_add_instructions(q)
print('r')
print(r)
s = sanity(r)
#parse_explain(s)
onesharp(s,['##'])

['1##', '1#']
[['0', '1##'], ['1', '1#']]
q below
[['0', '1##'], ['1', '1#']]
r
[['0', '1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1##1##1##1###'], ['1', '1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###']]
The input 1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1##1##1##1###1 is not a valid 1# program.
It is not the concatenation of a sequence of instructions in the language.
So what you are asking for is undefined.


'undefined'

In [108]:
numbered(diag)

[['0', '1'],
 ['1', '#'],
 ['2', '#'],
 ['3', '#'],
 ['4', '#'],
 ['5', '#'],
 ['6', '1'],
 ['7', '1'],
 ['8', '1'],
 ['9', '1'],
 ['10', '1'],
 ['11', '1'],
 ['12', '1'],
 ['13', '1'],
 ['14', '1'],
 ['15', '1'],
 ['16', '1'],
 ['17', '#'],
 ['18', '#'],
 ['19', '#'],
 ['20', '1'],
 ['21', '1'],
 ['22', '1'],
 ['23', '1'],
 ['24', '1'],
 ['25', '1'],
 ['26', '#'],
 ['27', '#'],
 ['28', '#'],
 ['29', '1'],
 ['30', '1'],
 ['31', '#'],
 ['32', '#'],
 ['33', '1'],
 ['34', '1'],
 ['35', '1'],
 ['36', '#'],
 ['37', '1'],
 ['38', '1'],
 ['39', '1'],
 ['40', '#'],
 ['41', '#'],
 ['42', '1'],
 ['43', '1'],
 ['44', '1'],
 ['45', '#'],
 ['46', '#'],
 ['47', '1'],
 ['48', '1'],
 ['49', '1'],
 ['50', '1'],
 ['51', '1'],
 ['52', '1'],
 ['53', '1'],
 ['54', '#'],
 ['55', '#'],
 ['56', '#'],
 ['57', '#'],
 ['58', '1'],
 ['59', '1'],
 ['60', '#'],
 ['61', '1'],
 ['62', '1'],
 ['63', '1'],
 ['64', '#'],
 ['65', '1'],
 ['66', '1'],
 ['67', '1'],
 ['68', '#'],
 ['69', '#'],
 ['70', '1'],
 ['71', '1'],
 [

In [50]:
int('44')

44